In [69]:
import nltk
#nltk.download('stopwords')
#nltk.download()

In [70]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
import re
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix,accuracy_score, classification_report
from nltk.tokenize import word_tokenize

In [71]:
import pandas as pd
df = pd.read_csv('//Users//omniaelmenshawy//Desktop///dataset.csv', encoding = 'utf8')
df.head()

,الله يشافيك و يعافيك من كل داء وهامة أخي سيد الوكيل,1
0,حلوه اوى,1
1,﻿حقا,1
2,صح جدا,1
3,﻿اه و النعمة,1
4,كلامك جميل ورائع,1


In [72]:
df.shape

(1978, 2)

In [73]:
df = df.rename(columns={'الله يشافيك و يعافيك من كل داء وهامة أخي سيد الوكيل': 'Sentences', '1': 'Sentiment'})
df

,Sentences,Sentiment
0,حلوه اوى,1
1,﻿حقا,1
2,صح جدا,1
3,﻿اه و النعمة,1
4,كلامك جميل ورائع,1
...,...,...
1973,البرنامج فاشل جدا,0
1974,ضيعت 7 دقايق من عمري بمشاهدة ازنخ و اتفه شخص ب...,0
1975,﻿ بكتب ع اوراق الشجر يقول موهبتي وانا صغير الل...,0
1976,هذا اللي بالنص من لجنة الحكم مفكر نفسو وديع ال...,0


In [74]:
df['Sentiment'].value_counts()

1    999
0    979
Name: Sentiment, dtype: int64

In [75]:
df['Sentences'] = df.fillna({'Sentences':''})

In [155]:
df = df.drop_duplicates()

In [156]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1949 entries, 0 to 1977
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Sentences  1949 non-null   object
 1   Sentiment  1949 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 45.7+ KB


# Cleaning:

In [157]:
from pandas.api.types import is_string_dtype
is_string_dtype(df['Sentences'])

True

In [158]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()

In [159]:
cv.fit(df.Sentences)

CountVectorizer()

In [160]:
x = cv.transform(df.Sentences).toarray()#.shape
x

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [161]:
cv.vocabulary_

{'حلوه': 2982,
 'اوي': 1811,
 'حقا': 2957,
 'صح': 3676,
 'جدا': 2810,
 'اه': 1769,
 'النعمه': 1560,
 'كلامك': 4407,
 'جميل': 2841,
 'وراءع': 5776,
 'خير': 3115,
 'الكلام': 1332,
 'قل': 4280,
 'دل': 3173,
 'ولا': 5926,
 'تنسو': 2727,
 'تبسمك': 2448,
 'وجه': 5729,
 'اخيك': 214,
 'صدقه': 3690,
 'ليتنا': 4691,
 'نستطيع': 5247,
 'الابتسامه': 538,
 'الاوقات': 671,
 'المءلمه': 1370,
 'بحب': 2068,
 'الناس': 1536,
 'الرايقه': 946,
 'الي': 1633,
 'بتضحك': 2023,
 'علي': 3978,
 'طول': 3810,
 'ابتسم': 32,
 'امام': 1655,
 'لتسرهم': 4512,
 'تجبر': 2474,
 'خواطرهم': 3111,
 'فتبسمك': 4126,
 'وجوههم': 5736,
 'الله': 1361,
 'انت': 1695,
 'حبيبي': 2902,
 'فالابتسامه': 4107,
 'تصنع': 2600,
 'السعاده': 1016,
 'جميله': 2842,
 'بنيه': 2341,
 'طيبه': 3815,
 'فقط': 4181,
 'معك': 5002,
 'حق': 2956,
 'لطيف': 4561,
 'البسمه': 705,
 'سر': 3458,
 'اسرار': 334,
 'الحياه': 874,
 'سنه': 3517,
 'وانتم': 5631,
 'طيبين': 3816,
 'كلمه': 4418,
 'بغير': 2230,
 'حروف': 2925,
 'افعال': 486,
 'تضاهيها': 2604,
 'اقوال': 506,
 'ت

In [162]:
len(cv.vocabulary_)

6557

In [163]:
x[0].max()

1

## Stop words, Punctioation and re cleaning::

In [164]:
print(stopwords.words('arabic'))

['إذ', 'إذا', 'إذما', 'إذن', 'أف', 'أقل', 'أكثر', 'ألا', 'إلا', 'التي', 'الذي', 'الذين', 'اللاتي', 'اللائي', 'اللتان', 'اللتيا', 'اللتين', 'اللذان', 'اللذين', 'اللواتي', 'إلى', 'إليك', 'إليكم', 'إليكما', 'إليكن', 'أم', 'أما', 'أما', 'إما', 'أن', 'إن', 'إنا', 'أنا', 'أنت', 'أنتم', 'أنتما', 'أنتن', 'إنما', 'إنه', 'أنى', 'أنى', 'آه', 'آها', 'أو', 'أولاء', 'أولئك', 'أوه', 'آي', 'أي', 'أيها', 'إي', 'أين', 'أين', 'أينما', 'إيه', 'بخ', 'بس', 'بعد', 'بعض', 'بك', 'بكم', 'بكم', 'بكما', 'بكن', 'بل', 'بلى', 'بما', 'بماذا', 'بمن', 'بنا', 'به', 'بها', 'بهم', 'بهما', 'بهن', 'بي', 'بين', 'بيد', 'تلك', 'تلكم', 'تلكما', 'ته', 'تي', 'تين', 'تينك', 'ثم', 'ثمة', 'حاشا', 'حبذا', 'حتى', 'حيث', 'حيثما', 'حين', 'خلا', 'دون', 'ذا', 'ذات', 'ذاك', 'ذان', 'ذانك', 'ذلك', 'ذلكم', 'ذلكما', 'ذلكن', 'ذه', 'ذو', 'ذوا', 'ذواتا', 'ذواتي', 'ذي', 'ذين', 'ذينك', 'ريث', 'سوف', 'سوى', 'شتان', 'عدا', 'عسى', 'عل', 'على', 'عليك', 'عليه', 'عما', 'عن', 'عند', 'غير', 'فإذا', 'فإن', 'فلا', 'فمن', 'في', 'فيم', 'فيما', 'فيه', 'فيها', '

In [165]:
stop_words = stopwords.words()

In [166]:
punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ''' + string.punctuation

In [167]:
def clean(text):
    
    RemovePunc = str.maketrans('', '', punctuations)
    text = text.translate(RemovePunc)
    
    #remove longation
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("گ", "ك", text)
    text = re.sub("ئ", "ء", text)
    
    

    text = ' '.join(word for word in text.split() if word not in stop_words)

    return text

## Stemmer:

In [168]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [169]:
def stem(text):
    clean_words = []
    for word in text.split():
        clean_words.append(ps.stem(word))
        
    return ' '.join(clean_words)

In [170]:
stem(df.Sentences[3])

'\ufeffاه و النعمه'

In [171]:
df['Sentences'] = df['Sentences'].apply(clean).apply(stem)
df.head(5)

<ipython-input-171-6d937f48ac93>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Sentences'] = df['Sentences'].apply(clean).apply(stem)


,Sentences,Sentiment
0,حلوه اوي,1
1,﻿حقا,1
2,صح جدا,1
3,﻿اه و النعمه,1
4,كلامك جميل وراءع,1


In [172]:
cv.fit(df.Sentences)

CountVectorizer()

In [173]:
x = cv.transform(df.Sentences).toarray()#.shape

In [174]:
len(cv.vocabulary_)

6557

In [175]:
print(7033-6557 , "Word deleted.")

476 Word deleted.


## Models:

In [176]:
x = df.Sentences
y = df.Sentiment
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size =.15, random_state=97)

## 1) LogisticRegression

In [177]:
pipe = make_pipeline(TfidfVectorizer(),LogisticRegression())
param_grid = {'logisticregression__C': [0.01, 0.1, 1, 10, 100]}

In [178]:
model = GridSearchCV(pipe, param_grid, cv=5)
model.fit(X_train,Y_train)
print('Done')

Done


In [179]:
LR_Prediction = model.predict(X_test)
print(f"Accuracy score is {accuracy_score(Y_test, LR_Prediction):.2f}")
print(classification_report(Y_test, LR_Prediction))

Accuracy score is 0.88
              precision    recall  f1-score   support

           0       0.89      0.87      0.88       155
           1       0.86      0.88      0.87       138

    accuracy                           0.88       293
   macro avg       0.88      0.88      0.88       293
weighted avg       0.88      0.88      0.88       293



## RandomForestClassifier

In [180]:
pipe = make_pipeline(TfidfVectorizer(),
                    RandomForestClassifier())

param_grid = {'randomforestclassifier__n_estimators':[10, 100, 1000],
             'randomforestclassifier__max_features':['sqrt', 'log2']}

In [181]:
RandomFO = GridSearchCV(pipe, param_grid, cv=5)
RandomFO.fit(X_train,Y_train)
print('Done')

Done


In [182]:
RF_Prediction = RandomFO.predict(X_test)
print(f"Accuracy score is {accuracy_score(Y_test, RF_Prediction):.2f}")
print(classification_report(Y_test, RF_Prediction))

Accuracy score is 0.86
              precision    recall  f1-score   support

           0       0.87      0.87      0.87       155
           1       0.86      0.86      0.86       138

    accuracy                           0.86       293
   macro avg       0.86      0.86      0.86       293
weighted avg       0.86      0.86      0.86       293



## SVC:

In [183]:
pipe = make_pipeline(TfidfVectorizer(),SVC())
param_grid = {'svc__kernel': ['rbf', 'linear', 'poly'],
             'svc__gamma': [0.1, 1, 10, 100],
             'svc__C': [0.1, 1, 10, 100]}

In [184]:
svc_model = GridSearchCV(pipe, param_grid, cv=3)
svc_model.fit(X_train, Y_train)
print('Done')

Done


In [185]:
SVC_Prediction = svc_model.predict(X_test)
print(f"Accuracy score is {accuracy_score(Y_test, SVC_Prediction):.2f}")
print(classification_report(Y_test, SVC_Prediction))

Accuracy score is 0.88
              precision    recall  f1-score   support

           0       0.90      0.87      0.89       155
           1       0.86      0.89      0.88       138

    accuracy                           0.88       293
   macro avg       0.88      0.88      0.88       293
weighted avg       0.88      0.88      0.88       293

